# Analysis Japanese candlesticks

#### ------------------------------
#### 1- get OHLC data 
#### 2- analysis big candlestick
#### ------------------------------

#### 1- get OHLC data 

In [74]:
import pandas as pd
df= pd.read_csv(r"C:\Users\Access\Documents\AI ROBOT\XAUUSD_Candlestick_1_M_BID_2020-1-1_2024-12-31.csv")
df=df.drop(columns='Volume')
import pandas as pd

# تحويل العمود الزمني إلى نوع datetime وتعيينه كمؤشر
df['Gmt time'] = pd.to_datetime(df['Gmt time'])
df.set_index('Gmt time', inplace=True)

# تحويل البيانات من 1 دقيقة إلى 5 دقائق
df_5m = df.resample('5T').agg({
    'Open': 'first',   # أول سعر افتتاح في الفاصل الزمني
    'High': 'max',     # أعلى سعر في الفاصل الزمني
    'Low': 'min',      # أقل سعر في الفاصل الزمني
    'Close': 'last'    # آخر سعر إغلاق في الفاصل الزمني
})

# عرض النتيجة
df_5m=df_5m.reset_index(drop=False) 


C:\Users\Access\AppData\Local\Temp\ipykernel_26228\117962932.py:11: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_5m = df.resample('5T').agg({


In [75]:
df_5m.columns

Index(['Gmt time', 'Open', 'High', 'Low', 'Close'], dtype='object')

#### 2- analysis big candlestick

In [76]:
import pandas as pd

# افترض أن df_5m هو DataFrame موجود مسبقًا
# قم باستيراد البيانات الخاصة بك هنا أو استخدم df_5m مباشرة إذا كان متاحًا
df = df_5m  # قم بحذف هذا السطر واستبداله بالطريقة التي تقرأ بها البيانات إذا لم يكن df_5m متاحًا

# تعريف دالة التحقق من الشمعة الكبيرة مقارنة بالظلال
def is_large_body(candle):
    body = abs(candle['Open'] - candle['Close'])
    lower_shadow = abs(candle['Low'] - min(candle['Open'], candle['Close']))
    upper_shadow = abs(candle['High'] - max(candle['Open'], candle['Close']))
    return (body*5) > (lower_shadow + upper_shadow)

# تعريف دالة التحقق من شمعة الهامر
def is_hammer(candle):
    body = abs(candle['Open'] - candle['Close'])
    lower_shadow = abs(candle['Low'] - min(candle['Open'], candle['Close']))
    upper_shadow = abs(candle['High'] - max(candle['Open'], candle['Close']))
    # تصحيح الشرط للهامر - الظل السفلي يجب أن يكون أكبر من الجسم بمقدار معين
    return lower_shadow > (2 * body) and upper_shadow < (0.5 * body)

# دالة حساب صفقة شراء باستخدام الهامر - تم تصحيح حساب وقف الخسارة والهدف
def calculate_buy_trade_hammer(row, df, tp_multiplier=1.5, sl_multiplier=0.5):
    idx = row.name  # الفهرس الزمني (Timestamp)
    if idx == df.index[0]:  # إذا كانت الشمعة الأولى، لا توجد صفقة
        return 0, None, None
    
    first_candle = df.loc[df.index[df.index.get_loc(idx) - 1]]  # الشمعة السابقة
    second_candle = row  # الشمعة الحالية
    
    # شرط إشارة الشراء: شمعة هابطة كبيرة تليها شمعة هامر
    if (is_large_body(first_candle) and 
        first_candle['Open'] > first_candle['Close'] and 
        is_hammer(second_candle) and 
        second_candle['Open'] < second_candle['Close'] and
        abs(first_candle['Open'] - first_candle['Close']) >= 0.5):
        
        # حساب حجم الشمعة الأولى (النقاط)
        points = abs(first_candle['Open'] - first_candle['Close'])
        
        # حساب وقف الخسارة: أقل من سعر الإغلاق للشمعة الثانية بمقدار محدد
        # عادة يكون عند أدنى نقطة للشمعة الثانية أو قريبًا منها
        stop_loss = min(second_candle['Low'], second_candle['Close'] - (points * sl_multiplier))
        
        # حساب الهدف: أعلى من سعر الإغلاق للشمعة الثانية بمضاعف المخاطرة
        take_profit = second_candle['Close'] + (tp_multiplier * (second_candle['Close'] - stop_loss))
        
        return 1, take_profit, stop_loss
    
    return 0, None, None


# تعديل دالة الهامر المقلوب (Shooting Star) - تصحيح الشرط والحساب
def is_shooting_star(candle):
    body = abs(candle['Open'] - candle['Close'])
    upper_shadow = abs(candle['High'] - max(candle['Open'], candle['Close']))
    lower_shadow = abs(candle['Low'] - min(candle['Open'], candle['Close']))
    # الظل العلوي يجب أن يكون أكبر من الجسم بمقدار معين
    return upper_shadow > (2 * body) and lower_shadow < (0.5 * body)

# تعديل دالة الصفقة للبيع - تصحيح حساب وقف الخسارة والهدف
def calculate_sell_trade_shooting_star(row, df, tp_multiplier=1.5, sl_multiplier=0.5):
    idx = row.name
    if idx == df.index[0]:
        return 0, None, None
    
    first_candle = df.loc[df.index[df.index.get_loc(idx) - 1]]
    second_candle = row
    
    # الشرط المعكوس: شمعة صاعدة كبيرة تليها شمعة Shooting Star
    if (is_large_body(first_candle) and 
        first_candle['Open'] < first_candle['Close'] and 
        is_shooting_star(second_candle) and 
        second_candle['Open'] > second_candle['Close'] and 
        abs(first_candle['Open'] - first_candle['Close']) >= 0.5):
        
        # حساب حجم الشمعة الأولى (النقاط)
        points = abs(first_candle['Open'] - first_candle['Close'])
        
        # حساب وقف الخسارة: أعلى من سعر الإغلاق للشمعة الثانية بمقدار محدد
        # عادة يكون عند أعلى نقطة للشمعة الثانية أو قريبًا منها
        stop_loss = max(second_candle['High'], second_candle['Close'] + (points * sl_multiplier))
        
        # حساب الهدف: أقل من سعر الإغلاق للشمعة الثانية بمضاعف المخاطرة
        take_profit = second_candle['Close'] - (tp_multiplier * (stop_loss - second_candle['Close']))
        
        return 1, take_profit, stop_loss
        
    return 0, None, None

# باقي الكود يبقى كما هو، مع تطبيق الدوال المصححة

# تطبيق الأعمدة الجديدة للصفقات الشرائية
df['buy_trade_hammer'], df['tp_buy_hammer'], df['sl_buy_hammer'] = zip(*df.apply(
    lambda row: calculate_buy_trade_hammer(row, df), axis=1))

# تحديد نوع الصفقة للشراء
df['trade_type_hammer'] = df.apply(lambda row: 1 if row['buy_trade_hammer'] == 1 else 0, axis=1)

# دمج أعمدة TP و SL للصفقات الشرائية
df['tp_hammer'] = df.apply(lambda row: row['tp_buy_hammer'] if row['trade_type_hammer'] == 1 else None, axis=1)
df['sl_hammer'] = df.apply(lambda row: row['sl_buy_hammer'] if row['trade_type_hammer'] == 1 else None, axis=1)

# دالة لتحديد حالة الصفقة الشرائية
def determine_trade_result_hammer(row, df):
    idx = row.name  # الفهرس الزمني (Timestamp)
    if row['trade_type_hammer'] == 1:  # صفقة شراء
        for i in range(df.index.get_loc(idx) + 1, len(df)):
            future_row = df.iloc[i]
            if future_row['High'] >= row['tp_hammer']:
                return 2  # رابحة
            elif future_row['Low'] <= row['sl_hammer']:
                return 1  # خاسرة
    return 0  # لا توجد نتيجة

# تطبيق دالة تحديد حالة الصفقة الشرائية
df['trade_result_hammer'] = df.apply(lambda row: determine_trade_result_hammer(row, df), axis=1)

# تطبيق الأعمدة الجديدة للصفقات البيعية
df['sell_trade_shooting'], df['tp_sell_shooting'], df['sl_sell_shooting'] = zip(*df.apply(
    lambda row: calculate_sell_trade_shooting_star(row, df), axis=1))

# تحديد نوع الصفقة للبيع
df['trade_type_shooting'] = df.apply(lambda row: -1 if row['sell_trade_shooting'] == 1 else 0, axis=1)

# دمج أعمدة TP و SL
df['tp_shooting'] = df.apply(lambda row: row['tp_sell_shooting'] if row['trade_type_shooting'] == -1 else None, axis=1)
df['sl_shooting'] = df.apply(lambda row: row['sl_sell_shooting'] if row['trade_type_shooting'] == -1 else None, axis=1)

# تعديل دالة تحديد نتيجة الصفقة للبيع
def determine_trade_result_shooting(row, df):
    idx = row.name
    if row['trade_type_shooting'] == -1:  # صفقة بيع
        for i in range(df.index.get_loc(idx) + 1, len(df)):
            future_row = df.iloc[i]
            if future_row['Low'] <= row['tp_shooting']:
                return 2  # رابحة
            elif future_row['High'] >= row['sl_shooting']:
                return 1  # خاسرة
    return 0  # لا توجد نتيجة

# تطبيق دالة النتائج
df['trade_result_shooting'] = df.apply(lambda row: determine_trade_result_shooting(row, df), axis=1)

# حساب عمود الرصيد النهائي بعد العمولة والربح/الخسارة
initial_balance = 1000  # رأس المال الابتدائي
final_balance = [initial_balance] * len(df)

for i in range(1, len(df)):
    previous_balance = final_balance[i-1]
    current_row = df.iloc[i]
    
    current_balance = previous_balance 
    
    # حساب نتائج الصفقات
    if current_row['trade_result_hammer'] == 2:  # صفقة هامر رابحة
        current_balance += current_balance * (0.03 - 0.00406)   # note : 0.00406 for 7$ commission , 0.03 1:3 r:r
    elif current_row['trade_result_hammer'] == 1:  # صفقة هامر خاسرة
        current_balance -= current_balance * (0.01 + 0.00406)   # note : 0.00406 for 7$ commission
        
    if current_row['trade_result_shooting'] == 2:  # صفقة شوتينج رابحة
        current_balance += current_balance * (0.03 - 0.00406)   # note : 0.00406 for 7$ commission , 0.03 1:3 r:r
    elif current_row['trade_result_shooting'] == 1:  # صفقة شوتينج خاسرة
        current_balance -= current_balance * (0.01 + 0.00406)   # note : 0.00406 for 7$ commission

    final_balance[i] = current_balance

df['final_balance'] = final_balance

# إعادة ترتيب الأعمدة
df = df[[
    'Open', 'High', 'Low', 'Close', 
    'trade_type_hammer', 'trade_result_hammer', 'tp_hammer', 'sl_hammer',
    'trade_type_shooting', 'trade_result_shooting', 'tp_shooting', 'sl_shooting',
    'final_balance'
]]

print('تم المعالجة بنجاح')

KeyboardInterrupt: 

In [55]:
import pandas as pd

# استخدام df_5m كمصدر البيانات
df = df_5m.copy()

# تأكد أن 'Gmt time' موجود وحوّله إلى datetime
df['Gmt time'] = pd.to_datetime(df['Gmt time'])

# استخراج التاريخ اليومي بدون الوقت
df['date'] = df['Gmt time'].dt.date

# جعل 'Gmt time' هو الفهرس
df.set_index('Gmt time', inplace=True)

# ====== دوال الكشف عن الشموع وأنواع الصفقات ======

def is_large_body(candle):
    body = abs(candle['Open'] - candle['Close'])
    lower_shadow = abs(candle['Low'] - min(candle['Open'], candle['Close']))
    upper_shadow = abs(candle['High'] - max(candle['Open'], candle['Close']))
    return (body * 5) > (lower_shadow + upper_shadow)

def is_hammer(candle):
    body = abs(candle['Open'] - candle['Close'])
    lower_shadow = abs(candle['Low'] - min(candle['Open'], candle['Close']))
    upper_shadow = abs(candle['High'] - max(candle['Open'], candle['Close']))
    return lower_shadow > (2 * body) and upper_shadow < (0.5 * body)

def is_shooting_star(candle):
    body = abs(candle['Open'] - candle['Close'])
    upper_shadow = abs(candle['High'] - max(candle['Open'], candle['Close']))
    lower_shadow = abs(candle['Low'] - min(candle['Open'], candle['Close']))
    return upper_shadow > (2 * body) and lower_shadow < (0.5 * body)

# ====== دوال صفقات الشراء والبيع ======

def calculate_buy_trade_hammer(row, df, tp_multiplier=1.5, sl_multiplier=0.5):
    idx = row.name
    if idx == df.index[0]:
        return 0, None, None

    first_candle = df.loc[df.index[df.index.get_loc(idx) - 1]]
    second_candle = row

    if (is_large_body(first_candle) and 
        first_candle['Open'] > first_candle['Close'] and 
        is_hammer(second_candle) and 
        second_candle['Open'] < second_candle['Close'] and 
        abs(first_candle['Open'] - first_candle['Close']) >= 0.5):

        points = abs(first_candle['Open'] - first_candle['Close'])
        stop_loss = min(second_candle['Low'], second_candle['Close'] - (points * sl_multiplier))
        take_profit = second_candle['Close'] + (tp_multiplier * (second_candle['Close'] - stop_loss))

        return 1, take_profit, stop_loss

    return 0, None, None

def calculate_sell_trade_shooting_star(row, df, tp_multiplier=1.5, sl_multiplier=0.5):
    idx = row.name
    if idx == df.index[0]:
        return 0, None, None

    first_candle = df.loc[df.index[df.index.get_loc(idx) - 1]]
    second_candle = row

    if (is_large_body(first_candle) and 
        first_candle['Open'] < first_candle['Close'] and 
        is_shooting_star(second_candle) and 
        second_candle['Open'] > second_candle['Close'] and 
        abs(first_candle['Open'] - first_candle['Close']) >= 0.5):

        points = abs(first_candle['Open'] - first_candle['Close'])
        stop_loss = max(second_candle['High'], second_candle['Close'] + (points * sl_multiplier))
        take_profit = second_candle['Close'] - (tp_multiplier * (stop_loss - second_candle['Close']))

        return 1, take_profit, stop_loss

    return 0, None, None

# ====== تطبيق الصفقات وتحديد الأنواع والنتائج ======

df['buy_trade_hammer'], df['tp_buy_hammer'], df['sl_buy_hammer'] = zip(*df.apply(
    lambda row: calculate_buy_trade_hammer(row, df), axis=1))

df['trade_type_hammer'] = df['buy_trade_hammer']
df['tp_hammer'] = df.apply(lambda row: row['tp_buy_hammer'] if row['trade_type_hammer'] == 1 else None, axis=1)
df['sl_hammer'] = df.apply(lambda row: row['sl_buy_hammer'] if row['trade_type_hammer'] == 1 else None, axis=1)

def determine_trade_result_hammer(row, df):
    idx = row.name
    if row['trade_type_hammer'] == 1:
        for i in range(df.index.get_loc(idx) + 1, len(df)):
            future_row = df.iloc[i]
            if future_row['High'] >= row['tp_hammer']:
                return 2
            elif future_row['Low'] <= row['sl_hammer']:
                return 1
    return 0

df['trade_result_hammer'] = df.apply(lambda row: determine_trade_result_hammer(row, df), axis=1)

df['sell_trade_shooting'], df['tp_sell_shooting'], df['sl_sell_shooting'] = zip(*df.apply(
    lambda row: calculate_sell_trade_shooting_star(row, df), axis=1))

df['trade_type_shooting'] = df['sell_trade_shooting'].apply(lambda x: -1 if x == 1 else 0)
df['tp_shooting'] = df.apply(lambda row: row['tp_sell_shooting'] if row['trade_type_shooting'] == -1 else None, axis=1)
df['sl_shooting'] = df.apply(lambda row: row['sl_sell_shooting'] if row['trade_type_shooting'] == -1 else None, axis=1)

def determine_trade_result_shooting(row, df):
    idx = row.name
    if row['trade_type_shooting'] == -1:
        for i in range(df.index.get_loc(idx) + 1, len(df)):
            future_row = df.iloc[i]
            if future_row['Low'] <= row['tp_shooting']:
                return 2
            elif future_row['High'] >= row['sl_shooting']:
                return 1
    return 0

df['trade_result_shooting'] = df.apply(lambda row: determine_trade_result_shooting(row, df), axis=1)

# ====== حساب الرصيد النهائي بعد كل صفقة ======

initial_balance = 1000
final_balance = [initial_balance]

for i in range(1, len(df)):
    previous_balance = final_balance[-1]
    current_row = df.iloc[i]
    current_balance = previous_balance

    if current_row['trade_result_hammer'] == 2:
        current_balance += previous_balance * (0.03 - 0.00406)
    elif current_row['trade_result_hammer'] == 1:
        current_balance -= previous_balance * (0.01 + 0.00406)

    if current_row['trade_result_shooting'] == 2:
        current_balance += previous_balance * (0.03 - 0.00406)
    elif current_row['trade_result_shooting'] == 1:
        current_balance -= previous_balance * (0.01 + 0.00406)

    final_balance.append(current_balance)

df['final_balance'] = final_balance

# ====== تحليلات بسيطة ======

num_days = df['date'].nunique()
total_hammer_trades = df['trade_type_hammer'].sum()
total_hammer_wins = (df['trade_result_hammer'] == 2).sum()
total_hammer_losses = (df['trade_result_hammer'] == 1).sum()
avg_hammer_per_day = total_hammer_trades / num_days if num_days else 0

total_shooting_trades = (df['trade_type_shooting'] == -1).sum()
total_shooting_wins = (df['trade_result_shooting'] == 2).sum()
total_shooting_losses = (df['trade_result_shooting'] == 1).sum()
avg_shooting_per_day = total_shooting_trades / num_days if num_days else 0

# طباعة النتائج
print("✅ تم التنفيذ بنجاح")
print(f"📊 عدد الأيام: {num_days}")
print(f"📈 إجمالي صفقات الهامر: {total_hammer_trades} (رابحة: {total_hammer_wins}, خاسرة: {total_hammer_losses})")
print(f"📉 إجمالي صفقات الهامر المقلوب: {total_shooting_trades} (رابحة: {total_shooting_wins}, خاسرة: {total_shooting_losses})")
print(f"🔥 متوسط صفقات الهامر يومياً: {avg_hammer_per_day:.2f}")
print(f"🔥 متوسط صفقات الشوتينج يومياً: {avg_shooting_per_day:.2f}")
print(f"💰 الرصيد النهائي: {df['final_balance'].iloc[-1]:.2f}")


✅ تم التنفيذ بنجاح
📊 عدد الأيام: 1827
📈 إجمالي صفقات الهامر: 1121 (رابحة: 449, خاسرة: 672)
📉 إجمالي صفقات الهامر المقلوب: 1078 (رابحة: 428, خاسرة: 650)
🔥 متوسط صفقات الهامر يومياً: 0.61
🔥 متوسط صفقات الشوتينج يومياً: 0.59
💰 الرصيد النهائي: 42102.16
